In [2]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from ipywidgets import widgets

In [ ]:
from tensorflow.keras.datasets import cifar10
# This gives you everything (images and labels) in 1 second
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [4]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
plt.figure(figsize=(12, 5))

# Loop through each of the 10 classes
for i in range(10):
    # Find the index of the first image belonging to class 'i'
    index = np.where(y_train == i)[0][0]

    # Select the image
    img = x_train[index]

    # Create a subplot
    plt.subplot(2, 5, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(img)
    plt.xlabel(class_names[i])

plt.show()

In [ ]:
# --- Training Set Counts ---
unique_train, counts_train = np.unique(y_train, return_counts=True)
train_counts = dict(zip(class_names, counts_train))

print("--- Training Set ---")
for name, count in train_counts.items():
    print(f"Total training {name} images: {count}")

print("\n" + "-"*20 + "\n")

# --- Test Set Counts ---
unique_test, counts_test = np.unique(y_test, return_counts=True)
test_counts = dict(zip(class_names, counts_test))

print("--- Test Set ---")
for name, count in test_counts.items():
    print(f"Total test {name} images: {count}")

In [ ]:
plt.figure(figsize=(10, 4))
plt.bar(test_counts.keys(), test_counts.values(), color='skyblue')
plt.title('Distribution of Classes in Test Set')
plt.xlabel('Class Name')
plt.ylabel('Number of Images')
plt.xticks(rotation=45)
plt.show()

In [8]:
model = tf.keras.models.Sequential([
    tf.keras.Input(shape = (32, 32, 3)),
    tf.keras.layers.Rescaling(1./255),
    # Convolution and Pooling layers
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(128, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation = 'relu'),
    # 10 output neurons. It will contain a value from 0-9
    tf.keras.layers.Dense(10, activation = 'softmax')
])

In [ ]:
model.summary()

In [10]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate= 0.0001),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics = ['accuracy']
)

In [ ]:
# 1. Create the Training Dataset
# We use from_tensor_slices to convert NumPy arrays to a TF Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))

# 2. Create the Validation (Test) Dataset
validation_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

BATCH_SIZE = 32

train_dataset = train_dataset.shuffle(buffer_size=50000).batch(BATCH_SIZE)
validation_dataset = validation_dataset.batch(BATCH_SIZE)

print("Dataset objects created successfully!")

In [12]:
SHUFFEL_BUFFER_SIZE = 50000
PREFETCH_BUFFER_SIZE = tf.data.AUTOTUNE
train_dataset_final = train_dataset.cache().shuffle(SHUFFEL_BUFFER_SIZE).prefetch(PREFETCH_BUFFER_SIZE)
validation_dataset_final = validation_dataset.cache().prefetch(PREFETCH_BUFFER_SIZE)

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',     # Watch the validation loss
    patience=2,             # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True # Revert the model to the weights from the best epoch
)

history = model.fit(
    train_dataset_final,
    epochs=45,
    validation_data=validation_dataset_final,
    verbose=2,
    callbacks=[early_stopping]
)

In [ ]:
from io import BytesIO
from IPython.display import display

# 1. Create the widgets
uploader = widgets.FileUpload(accept="image/*", multiple=True)
out = widgets.Output()

def file_predict(filename, file_stream, output_area):
    """
    Loads the uploaded image, resizes it to 32x32,
    and predicts the CIFAR-10 class.
    """
    # Load image and resize to 32x32 to match CIFAR-10 model input
    img = tf.keras.utils.load_img(file_stream, target_size=(32, 32))
    img_array = tf.keras.utils.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Create batch axis (1, 32, 32, 3)

    # Generate predictions
    predictions = model.predict(img_array, verbose=0)

    # Get the class with the highest probability
    class_idx = np.argmax(predictions[0])
    predicted_label = class_names[class_idx]
    confidence = 100 * np.max(predictions[0])

    with output_area:
        print(f"File: {filename} | Prediction: {predicted_label} ({confidence:.2f}%)")

def on_upload_change(change):
    uploaded_dict = change.new

    # Iterate through the dictionary values to get the file metadata
    for file_data in uploaded_dict.values():
        name = file_data['metadata']['name']
        content = file_data['content']

        file_predict(name, BytesIO(content), out)

# 2. Observe the value change and display
uploader.observe(on_upload_change, names='value')
display(uploader, out)

In [ ]:
def plot_loss_acc(history):
    '''Plots the training and validation loss and accuracy from a history object'''
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))

    fig, ax = plt.subplots(1,2, figsize=(12, 6))
    ax[0].plot(epochs, acc, 'bo', label='Training accuracy')
    ax[0].plot(epochs, val_acc, 'b', label='Validation accuracy')
    ax[0].set_title('Training and validation accuracy')
    ax[0].set_xlabel('epochs')
    ax[0].set_ylabel('accuracy')
    ax[0].legend()

    ax[1].plot(epochs, loss, 'bo', label='Training Loss')
    ax[1].plot(epochs, val_loss, 'b', label='Validation Loss')
    ax[1].set_title('Training and validation loss')
    ax[1].set_xlabel('epochs')
    ax[1].set_ylabel('loss')
    ax[1].legend()

    plt.show()

plot_loss_acc(history)

In [ ]:
from sklearn.metrics import classification_report

# 1. Get the model's predictions (probabilities) for the test set
# We use x_test directly or your validation_dataset_final
print("Generating predictions...")
predictions = model.predict(x_test, verbose=1)

# 2. Convert probabilities to class indices (0-9)
y_pred = np.argmax(predictions, axis=1)

# 3. Flatten y_test if necessary (it is usually shape (10000, 1))
y_true = y_test.flatten()

# 4. Generate and print the report
report = classification_report(y_true, y_pred, target_names=class_names)
print("\nClassification Report:\n")
print(report)

In [17]:
model.save('cifar10_model.keras')